In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm.notebook import tqdm
%matplotlib inline

In [ ]:
!pip install modelzoo-client[transformers]

     |████████████████████████████████| 46 kB 2.6 MB/s 
     |████████████████████████████████| 82 kB 1.4 MB/s 
     |████████████████████████████████| 789 kB 51.1 MB/s 
     |████████████████████████████████| 4.0 MB 56.3 MB/s 
     |████████████████████████████████| 6.6 MB 49.0 MB/s 
     |████████████████████████████████| 77 kB 9.0 MB/s 
     |████████████████████████████████| 895 kB 59.5 MB/s 
     |████████████████████████████████| 596 kB 58.9 MB/s 
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803699 sha256=ecec556a41fed9adb992232c606e17ceeb762d50b3b5eedb9e17881fb752c59d
  Stored in directory: /root/.cache/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      S

In [ ]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification, AdamW
from transformers import get_cosine_schedule_with_warmup

model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-small-v2-discriminator')
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

Downloading:   0%|          | 0.00/486 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
badwords = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/project/badwords.csv")

In [ ]:
badwords_data = badwords[['문장', '악플욕설']]

In [ ]:
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
from torch import torch
from sklearn.model_selection import train_test_split

def dataSplit(dataset, y_label):

  X_train, X_val= train_test_split(dataset, test_size = 0.2, stratify = dataset[y_label], random_state =427)

  return X_train, X_val

In [ ]:
X_train, X_val = dataSplit(badwords_data, '악플욕설')

In [ ]:
class LoadDataset(Dataset):
    def __init__(self, df, tk):
        self.df = df
        self.tokenizer = tk
        
    def __len__(self):
        return len(self.df)
  
    def __getitem__(self, idx):
        row = self.df.iloc[idx, :].values
        # target이 없는경우 (즉, 문장만 입력된 경우)
        if len(row) <= 1:
            text = row[0]

            inputs = self.tokenizer(
                text, 
                return_tensors='pt',
                truncation=True,
                max_length=50,
                pad_to_max_length=True,
                add_special_tokens=True
                )
            
            input_ids = inputs['input_ids'][0]
            attention_mask = inputs['attention_mask'][0]

            return input_ids, attention_mask     
            
        # target이 있는 경우 (원래 코드)
        else:
            text = row[0]
            y = row[1]

            inputs = self.tokenizer(
                text, 
                return_tensors='pt',
                truncation=True,
                max_length=50,
                pad_to_max_length=True,
                add_special_tokens=True
                )
            
            input_ids = inputs['input_ids'][0]
            attention_mask = inputs['attention_mask'][0]

            return input_ids, attention_mask, y

In [ ]:
train_set = LoadDataset(X_train, tokenizer)
val_set = LoadDataset(X_val, tokenizer)

In [ ]:
epochs = 1 #epoch 10으로 설정하고 훈련시킨 후 1로 설정해서 한번씩 돌리고 결과 보기
batch_size = 128
warmup_ratio=0.1
t_total = len(train_set) * epochs
optimizer = AdamW(model.parameters(), lr=1e-4, eps = 1e-8)
train_loader = DataLoader(train_set, batch_size=batch_size)
test_loader = DataLoader(val_set, batch_size=batch_size)
loss_f = nn.BCELoss()

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=t_total)

In [ ]:
model.classifier.out_proj =  nn.Sequential( nn.Linear(256, 1),
                                           nn.Sigmoid() )

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from sklearn.metrics import roc_auc_score
losses = []
accuracies = []

for i in range(epochs):
    total_loss = 0.0
    correct = 0
    batches = 0
    score_list = []
    total = 0
    batches = 0


    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_pred = model(input_ids_batch, attention_mask=attention_masks_batch).logits.reshape(-1)
        loss = loss_f(y_pred.type(torch.FloatTensor), y_batch.type(torch.FloatTensor))
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        try:
            score = roc_auc_score(y_batch.tolist(), y_pred.tolist())
            score_list.append(score)
        except: pass
        batches += 1
        if batches % 30 == 0:
            print("Batch Loss:", total_loss, "ROC_AUC:",np.mean(score_list))


    print("Train Loss:", total_loss, "epoch roc_auc:", np.mean(score_list))

  0%|          | 0/50 [00:00<?, ?it/s]

Batch Loss: 2.4844261333346367 ROC_AUC: 0.9960227905941396
Train Loss: 4.000156873837113 epoch roc_auc: 0.9960021899975874


In [ ]:
from sklearn.metrics import roc_auc_score
model.eval()
valid_loader = DataLoader(val_set, batch_size=batch_size)
score_list = []
for input_ids_batch, attention_masks_batch, y_batch in tqdm(valid_loader):
    y_pred = model(input_ids_batch, attention_mask=attention_masks_batch).logits.reshape(-1)
#    print(y_pred)
    try:
        score = roc_auc_score(y_batch.tolist(), y_pred.tolist())
        score_list.append(score)
    except: pass


print("epoch roc_auc:", np.mean(score_list))

  0%|          | 0/13 [00:00<?, ?it/s]

epoch roc_auc: 0.821451880361284


In [ ]:
PATH = "/content/gdrive/MyDrive/badwords_koelectra_model.pth"

torch.save({
            'epoch': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, PATH)